# Scalable Diffusion Models with Transformer (DiT)

This notebook samples from pre-trained DiT models. DiTs are class-conditional latent diffusion models trained on ImageNet that use transformers in place of U-Nets as the DDPM backbone. DiT outperforms all prior diffusion models on the ImageNet benchmarks.

[Project Page](https://www.wpeebles.com/DiT) | [HuggingFace Space](https://huggingface.co/spaces/wpeebles/DiT) | [Paper](http://arxiv.org/abs/2212.09748) | [GitHub](github.com/facebookresearch/DiT)

# 1. Setup / Установка нужных библиотек

We recommend using GPUs (Runtime > Change runtime type > Hardware accelerator > GPU)

Рекомендуется использовать GPU, в противном случае все будет ужасно медленным.

In [ ]:
# the requirements file contains a list of libraries that I use on my home PC
# в файле requirements представлен список библиотек, который используя я у себя на домашнем пк

# !pip install -r requirements.txt

In [ ]:
!git clone https://github.com/IlyaAb/DiT.git

In [ ]:
#for collab

!pip install diffusers==0.18.2
!pip install accelerate
!pip install torchinfo
!pip install timm

In [ ]:
#most of these libraries are needed only in case of experiments in a laptop, all important steps are done by accessing the .py files through the terminal, where the necessary imports are implemented
#большинство из этих библиотек нужны только в случае экспериментов в ноутбуке, все важные шаги осуществляются путем обращения к .py файлам через терминал, где нужные импорты реалзиованы

import DiT, os
os.chdir(r'DiT')

import torch
from torchvision.utils import save_image
import torchinfo
from diffusion import create_diffusion
from diffusers.models import AutoencoderKL
from download import find_model
from models import DiT_XL_2
import shutil
from PIL import Image
from IPython.display import display
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("GPU not found. Using CPU instead.")

In [ ]:
#show model arch. how much params

image_size = 256
latent_size = int(image_size) // 8
model = DiT_XL_2(input_size=latent_size)

torchinfo.summary(model)

# 2. Loading the dataset. Here is an example with an already existing drone dataset from /
# Загрузка датасета. Здесь представлен пример с уже имеющимся датасетом дронов из

https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav

In [ ]:
#saving only pictures in a separate folder, when creating a folder, you need to remember about the hierarchy, each class has a separate folder
#сохранение в отдельную папку только картинок, при создании папки нужно помнить об иерархии, каждому классу отдельная папка
source = '...drone_dataset_yolo\dataset_txt'
for i in os.listdir(source):
    if 'jpg' in i:
        shutil.copy(f'{source}\{i}', '...\drones_for_dit\drones')

# 3. Extracting features from dataset /
# Извлечение фич из данных

Создание фич с помощью энкодера из AutoencoderKL

In [ ]:
#vae arch
vae_model = "stabilityai/sd-vae-ft-ema"
vae = AutoencoderKL.from_pretrained(vae_model) #.to(device)

In [ ]:
# this part of the model is not trained, first the encoder is taken to extract the features, and at the end the decoder is taken to get the pictures again
# это часть модели не обучается, сначалаб берется жнкодер для извленчение фич, а в конце берется декодер, чтобы снова получить картинки
torchinfo.summary(vae)

In [ ]:
!torchrun \
--nnodes=1 \
--nproc_per_node=1 extract_features.py \
--data-path ...\drones_for_dit \
--features-path ...\dit_features2 \
--image-size 256 \
--global-batch-size 256 \
--global-seed 0 \
--vae ema \
--num-workers 0

# 4.Training the model on the received features from the encoder
# Тренировка модели на полученных фичах из энкодера

In [ ]:
#for finetune must go to train.py and uncomment string which change y_embedder dimension in loading weights

In [ ]:
!accelerate launch \
--mixed_precision fp16 train.py \
--feature-path ...\dit_features \
--results-dir ...\dit_weights \
--model DiT-S/2 \
--image-size 256 \
--num-classes 1 \
--epochs=10 \
--global-batch-size=16 \
--global-seed 0 \
--num-workers 0 \
--log-every=25 \
--ckpt-every=100

# 5.Sampling

In [ ]:
#sampling several imgs from pretrained XL model

!python sample.py \
--model DiT-XL/2 \
--vae ema \
--image-size=256 \
--num-classes=1000 \
--cfg-scale 8 \
--num-sampling-steps=10 \
--seed 0 \
--save-result-path /content/ \
--list-classes "895, 0, 42" #define number and labels of classes /определяет кол-во, и классы сэмплированных изображений
#--ckpt ...\DiT-XL-2-256x256.pt #если здесь не указывать ничего то будет автоматически скачаны веса XL модели

In [ ]:
#link for pretrained DiT_S model for generating drones imgs (not good qual)
# https://drive.google.com/file/d/1tW5hSCysitZ7tKOj5Ko9k4PdX1Pk6uZ7/view?usp=sharing

In [ ]:
#example of sampling from downloaded weigths

!python sample.py \
--model DiT-S/2 \
--vae ema \
--image-size=256 \
--num-classes=1 \
--cfg-scale 8 \
--num-sampling-steps=110 \
--seed 0 \
--save-result-path /content/ \
--list-classes "0, 0, 0" \
#define number and labels of classes (there are only one classes (drones))/определяет кол-во, и классы сэмплированных изображений
--ckpt /content/dit_s_drone_pretrain_35000 #если здесь не указывать ничего то будет автоматически скачаны веса XL модели

In [ ]:
#sampling many imgs, very useful if u have more than one gpu, generate npz file for FID metric

!torchrun sample_ddp.py \
--model DiT-XL/2 \
--vae ema \
--sample-dir /path/to/generated/data \
--per-proc-batch-size 32 \
--num-fid-sample 400 \
--image-size 265 \
--num-classes 1 \
--cfg-scale 7 \
--num-sampling-steps 250 \
--global-seed 42 \
--ckpt /path/to/your/trained/models/weights